In [44]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

In [45]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

In [46]:
train = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3360 files belonging to 6 classes.
Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


In [47]:
inputs = keras.Input(shape=(100, 100, 3))
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(6, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_95 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_45 (MaxPoolin  (None, 49, 49, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_96 (Conv2D)          (None, 47, 47, 64)        18496     
                                                                 
 conv2d_97 (Conv2D)          (None, 45, 45, 64)        36928     
                                                                 
 max_pooling2d_46 (MaxPoolin  (None, 22, 22, 64)       0         
 g2D)                                                     

In [48]:
history = model.fit(
  train.cache(),
  epochs=40,
  validation_data=val.cache()
)

Epoch 1/40


2022-08-01 06:29:43.229151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 2.3983 - accuracy: 0.1774

2022-08-01 06:29:50.649776: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 8s 70ms/step - loss: 2.3983 - accuracy: 0.1774 - val_loss: 1.9648 - val_accuracy: 0.1667
Epoch 2/40
105/105 [==============================] - 7s 67ms/step - loss: 1.9049 - accuracy: 0.2521 - val_loss: 1.5474 - val_accuracy: 0.3028
Epoch 3/40
105/105 [==============================] - 7s 66ms/step - loss: 1.3246 - accuracy: 0.4872 - val_loss: 0.8726 - val_accuracy: 0.6528
Epoch 4/40
105/105 [==============================] - 7s 66ms/step - loss: 0.9525 - accuracy: 0.6333 - val_loss: 0.8222 - val_accuracy: 0.7306
Epoch 5/40
105/105 [==============================] - 7s 66ms/step - loss: 0.7233 - accuracy: 0.7560 - val_loss: 0.5448 - val_accuracy: 0.8083
Epoch 6/40
105/105 [==============================] - 7s 66ms/step - loss: 0.4861 - accuracy: 0.8357 - val_loss: 0.4998 - val_accuracy: 0.8611
Epoch 7/40
105/105 [==============================] - 7s 66ms/step - loss: 0.4542 - accuracy: 0.8842 - val_loss: 0.4426 - val_accuracy: 0.8722
Epoch 8/40

In [52]:
model.evaluate(test)

23/23 [==============================] - 1s 21ms/step - loss: 1.0162 - accuracy: 0.9597


[1.0161926746368408, 0.9597222208976746]

In [53]:
keras.models.save_model(model, "./models/zenodo_1.h5")